In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import copy

In [ ]:
# GLOBAL: colors etc...
clrs        = ['tab:red', 'tab:blue', 'tab:green']
methodnames = ['FEM','DEC', 'REG'] 
error_sign  = r'$e_m$'
error_angle = r'$e_{\theta}$'
metric_mean = r'$m_{\theta}$'

metric_names = {'mean_topdiff' : error_sign, 'mean': metric_mean, 'mean_arccos' : error_angle}
markers = ['.', 'x']

In [ ]:
def hist_average(xs, ys, ints):
        stps = np.linspace( xs.min(), xs.max(), ints)
        stp_inds = [ ((stps[i] <= xs) & (xs < stps[i+1])) for i in range(len(stps) - 2)]
        stp_inds.append((stps[-1] <= xs))
        xs = (stps[:-1] + stps[1:])/2.
        ys = [ys[inds].mean() for inds in stp_inds]
        return xs, ys
    
def moving_average(xs, ys, width=100):
                idxs = np.argsort(xs)
                rst = width - len(idxs)%width
                pd_left  = rst // 2
                pd_right = rst // 2 + int(rst % 2 != 0)
                xs = np.concatenate([np.ones(pd_left) * xs[idxs[0]], xs[idxs], np.ones(pd_right) * xs[idxs[-1]]])
                ys = np.concatenate([np.ones(pd_left) * ys[idxs[0]], ys[idxs], np.ones(pd_right) * ys[idxs[-1]]])
                xs = xs.reshape(-1, width).mean(axis=1)
                ys = ys.reshape(-1, width).mean(axis=1)
                return xs, ys

In [ ]:
def load_heat_gradients(csv_path):
    df = pd.read_csv(csv_path)  
    
    anchors = df.values[:, :3]
    hgs_fem = df.values[:, 3:6]
    hgs_dec = df.values[:, 6:9]
    
    hgs_reg = None
    if df.shape[1] == 12:
        hgs_reg = df.values[:, 9:12]
    
    anchor_norms = np.linalg.norm(anchors, axis=1)
    origin_ind = np.nonzero(anchor_norms == 0)
    anchor_norms[origin_ind] = 1.
    anchors_normed = anchors / anchor_norms[:,None]
    
    hgs_fem_normed = - hgs_fem / np.linalg.norm(hgs_fem, axis=1, keepdims=True)
    hgs_dec_normed = - hgs_dec / np.linalg.norm(hgs_dec, axis=1, keepdims=True)
    if hgs_reg is not None:
        hgs_reg_normed = -hgs_reg / np.linalg.norm(hgs_reg, axis=1, keepdims=True)
        
    dot_products_fem = np.einsum('ij,ij->i', anchors_normed, hgs_fem_normed)
    dot_products_dec = np.einsum('ij,ij->i', anchors_normed, hgs_dec_normed)
    
    if hgs_reg is not None:
        dot_product_reg = np.einsum('ij,ij->i', anchors_normed, hgs_reg_normed)
    
    # names = ['fem', 'dec']
    names = methodnames[:2]
    products = [dot_products_fem, dot_products_dec]
    if hgs_reg is not None:
        # names.append('reg')
        names = methodnames
        products.append(dot_product_reg)
    
    return names, anchors, products

# plotting functions
def plot_fun_scatter(ax, anchor_norms, products, c, applfun=lambda x:x, ints=100, histmean=True, removezeropoint=True, addmean='hist'):
    if addmean is not None:
        if (addmean == 'hist'):
            xs, ys = hist_average(anchor_norms, products, ints)
        else:
            xs, ys = moving_average(anchor_norms, products)
        ax.plot(xs, ys, c='black', alpha=0.8)
    if removezeropoint:
        ax.scatter(anchor_norms[products != 0], applfun(products[products != 0]), s=0.5, c=c)
    else:
        ax.scatter(anchor_norms, applfun(products), s=0.5, c=c)

def compare_heat_gradients_cv(csv_prefix, plot=False, outpt=True, plothist=False, saveprefix=None, scatterangle=False, addzeroline=False, removezeropoint=True):
    
    metric_functions = {
        'mean'                 : lambda x : x.mean(),
        'mean_topdiff'         : lambda x : (1 - x).mean(),
        'max_topdiff'          : lambda x : (1 - x).max(),
        'mean_topdiff_squared' : lambda x : ((1 - x) ** 2).mean(),
        'mean_arccos'          : lambda x : np.arccos(x).mean(),
        'max_arccos'           : lambda x : np.arccos(x).max()
    }
    
    if outpt:
        print('CSV PREFIX: {}'.format(csv_prefix))
    
    csvp_cells = csv_prefix + '_cells.csv'
    csvp_verts = csv_prefix + '_vertices.csv'
    
    names, anchors_cells, products_cells = load_heat_gradients(csvp_cells)
    names, anchors_verts, products_verts = load_heat_gradients(csvp_verts)
    
    # join metrics in a df
    value_dict = {'anchor'    : ['cell'] * len(products_cells) + ['vertex'] * len(products_verts), 
                  'laplacian' : names * 2 }
    for m_k, m in metric_functions.items():
        value_dict[m_k] = [m(pc) for pc in products_cells] + [m(pv) for pv in products_verts]
                  
    reg_included = False
    if (len(products_cells) == 3):
        reg_included = True

    df = pd.DataFrame(value_dict)
    
    if outpt:
        display(df.set_index(['anchor', 'laplacian']))
    
    if plot:
    
        anchor_norms_cells = np.linalg.norm(anchors_cells, axis=1)
        anchor_norms_verts = np.linalg.norm(anchors_verts, axis=1)

        # PLOTS
        rows = 2
        cols = 2 if not reg_included else 3
        fig, axs = plt.subplots(rows, cols, figsize=(10, 5), sharex='col', sharey='row')
        
        nbins = 500
        scatteropts = {'s': 0.3, 'alpha':0.8}
        
        # switch between angle / cos(angle) formats
        if scatterangle:
            applfun = lambda x:np.arccos(x)
            ylabel  = r'$\theta$' 
            ylims   = [0., np.pi]
            metric_shown = 'mean_arccos'
        else:
            applfun = lambda x:x
            ylabel  = r'cos($\theta$)'
            ylims   = [-1, 1]
            metric_shown = 'mean'
            
        # add y=0 line
        for axrows in axs:
            for ax in axrows:
                ax.axhline(y=0, c='black', lw=0.5, ls='--')
                
        # switch betwen hist and scatter plot:
        if plothist:
            # dont use this! it looks awful
            plot_fun = lambda ax, anchor_norms, products, c : ax.hist2d(anchor_norms, applfun(products), bins=nbins, norm=mpl.colors.LogNorm())
        else:
            plot_fun = lambda ax, anchor_norms, products, c : plot_fun_scatter(ax, anchor_norms, products, c, removezeropoint=removezeropoint, applfun=applfun)
#             if removezeropoint:
#                 plot_fun = lambda ax, anchor_norms, products, c : ax.scatter(anchor_norms[products != 0], applfun(products[products != 0]), s=0.5, c=c)
#             else:
#                 plot_fun = lambda ax, anchor_norms, products, c : ax.scatter(anchor_norms, applfun(products), s=0.5, c=c)
        
        # cell-based FEM
        mtdval = df[(df['anchor'] == 'cell' )&(df['laplacian'] == names[0])][metric_shown].iloc[0]
        axs[0, 0].set_title('Cells, {} ({}:{:5f})'.format(names[0], metric_names[metric_shown], mtdval))
        plot_fun(axs[0,0], anchor_norms_cells, products_cells[0], clrs[0])
        axs[0, 0].set_ylim(ylims)
        axs[0, 0].set_ylabel(ylabel)
        
        # cell-based DEC
        mtdval = df[(df['anchor'] == 'cell' )&(df['laplacian'] == names[1])][metric_shown].iloc[0]
        axs[0, 1].set_title('Cells, {} ({}:{:5f})'.format(names[1], metric_names[metric_shown], mtdval))
        plot_fun(axs[0,1], anchor_norms_cells, products_cells[1], clrs[1])
        axs[0, 1].set_ylim(ylims)
        
        # vert-based FEM
        mtdval = df[(df['anchor'] == 'vertex' )&(df['laplacian'] == names[0])][metric_shown].iloc[0]
        axs[1, 0].set_title('Verts, {} ({}:{:5f})'.format(names[0], metric_names[metric_shown], mtdval))
        plot_fun(axs[1,0], anchor_norms_verts, products_verts[0], clrs[0])
        axs[1, 0].set_ylim(ylims)
        axs[1, 0].set_ylabel(ylabel)
        axs[1, 0].set_xlabel('anchor distance to origin')
        
        # vert-based DEC
        mtdval = df[(df['anchor'] == 'vertex' )&(df['laplacian'] == names[1])][metric_shown].iloc[0]
        axs[1, 1].set_title('Verts, {} ({}:{:5f})'.format(names[1], metric_names[metric_shown], mtdval))
        plot_fun(axs[1,1], anchor_norms_verts, products_verts[1], clrs[1])
        axs[1, 1].set_ylim(ylims)
        axs[1, 1].set_xlabel('anchor distance to origin')

        if (reg_included):
            mtdval = df[(df['anchor'] == 'cell')&(df['laplacian'] == names[2])][metric_shown].iloc[0]
            axs[0, 2].set_title('Cells, {} ({}:{:5f})'.format(names[2], metric_names[metric_shown], mtdval))
            plot_fun(axs[0, 2], anchor_norms_cells, products_cells[2], clrs[2])
            axs[0, 2].set_ylim(ylims)
            
            mtdval = df[(df['anchor'] == 'vertex')&(df['laplacian'] == names[2])][metric_shown].iloc[0]
            axs[1, 2].set_title('Verts, {} ({}:{:5f})'.format(names[2], metric_names[metric_shown], mtdval))
            plot_fun(axs[1, 2], anchor_norms_verts, products_verts[2], clrs[2])
            axs[1, 2].set_ylim(ylims)
            axs[1, 2].set_xlabel('anchor distance to origin')

        plt.tight_layout()
        if saveprefix is not None:
            plt.savefig(saveprefix + '.png'.format(), dpi=300)
        plt.show()
        fig.clf()
    
    return df

def plot_ifo_val(df, by_val, metrics=None, saveprefix=None):
    lss  = ['-', '--']
    
    index_cols = ['anchor', 'laplacian', by_val]
    
    axis_descriptors = {'t' : r'$log_{10}(t)$'}
    
    if metrics is None:
        metrics = [m for m in df.keys() if m not in index_cols]
    
    print(index_cols)
    
    if 'rep_ind' in df.keys():
        plotdf = df.groupby(index_cols).mean()
        display(plotdf)
        plotdf = plotdf.reset_index()
    else:
        plotdf = df
        
    display(plotdf.head())
    
    for m in metrics:
        fig, ax = plt.subplots(1, 1, figsize=(10, 5))
        for a_idx, an in enumerate(plotdf['anchor'].unique()): 
            for l_idx, ln in enumerate(plotdf['laplacian'].unique()):
                plotdf[(plotdf['anchor'] == an) & (plotdf['laplacian'] == ln)].plot(x=by_val, y=m, label='{} ({})'.format(ln, an), ax=ax, c=clrs[l_idx], ls=lss[a_idx], alpha=0.8)
        
        ax.set_ylabel(metric_names[m] if m in metric_names.keys() else m)
        if (by_val in list(axis_descriptors.keys())):
            ax.set_xlabel(axis_descriptors[by_val])
        
        if saveprefix is not None:
            plt.savefig(saveprefix + m + '.png', dpi=300)
        plt.show()
        
def plot_scatters_by_time(csv_prefixes, timesteps, vert_switch=True, saveprefix=None, addmean=None):
    
    assert(len(csv_prefixes) == len(timesteps)), "cscv prefixes and timesteps do not match!"
    csv_paths = [csv_p + '_vertices.csv' if vert_switch else '_cells.csv' for csv_p in csv_prefixes]

    # general plotting options
    applfun = lambda x:x
    ylims   = [-1, 1]
    metric_shown = 'mean'
    xlabel  = 'anchor dist to origin'
    ylabel  = metric_names[metric_shown]
        
    def config_ax(cax, addxlabel=False):
        cax.set_ylabel(ylabel)
        cax.set_ylim(ylims)
        if addxlabel:
            cax.set_xlabel(xlabel)
    
    rows = 3
    cols = len(timesteps)
    fig, axs = plt.subplots(rows, cols, figsize=(10, 7), sharex=True, sharey=True)
    for t_idx, timestep in enumerate(timesteps):
        names, anchors, products = load_heat_gradients(csv_paths[t_idx])
        anchor_norms = np.linalg.norm(anchors, axis=1)
        for (l_idx, name) in enumerate(names):
            cax = axs[t_idx, l_idx]
            mcs = '{:5f}'.format(products[l_idx].mean())
            cax.set_title('{} t=1e-{} ({}: {})'.format(name, timesteps[t_idx], metric_mean, mcs))
            plot_fun_scatter(cax, anchor_norms, products[l_idx], clrs[l_idx], addmean=addmean)
            config_ax(cax, t_idx==len(timesteps)-1)
            
    for axrows in axs:
        for ax in axrows:
            ax.axhline(y=0, c='black', lw=0.5, ls='--')
    
    plt.tight_layout()
    if saveprefix is not None:
        plt.savefig(saveprefix + '_meanscatters_3ts.png', dpi=300)
    plt.show()

# Stats IFO timestep t

In [ ]:
cellSize = 0.05
folderprefix = 'LA_hd_t1e'
expsteps = [-7, -5, -3]
csv_prefixes = ['../build/out/{}{}/plots/OriginSphere{}_-2.000000_R0_heatgradients'.format(folderprefix, t, cellSize) for t in expsteps]
save_prefix = '../build/out/{}{}/{}'.format(folderprefix, expsteps[0],cellSize)
plot_scatters_by_time(csv_prefixes, expsteps, ['angle'], save_prefix, addmean='hist')

In [ ]:
meandfs = []
expsteps = [-7, -6, -5, -4, -3, -2, -1, 0]
cellSize = 0.05
folderprefix = 'LA_hd_t1e'
for texp in expsteps:
    foldername = '{}{}'.format(folderprefix, texp)
    # foldername = 'KA_hd_t1e{}'.format(texp)
    folderpath = '../build/out/{}/plots'.format(foldername)
    meshesprefix = folderpath + '/OriginSphere{}_-2.000000_R0_heatgradients'.format(cellSize)
    meandfs.append(compare_heat_gradients_cv(meshesprefix, 
                                             scatterangle=False,
                                             plot=True,
                                             addzeroline=True,
                                             saveprefix='../build/out/{}{}/cs{}_t1e{}heatscatters'.format(folderprefix, expsteps[0], cellSize, texp)))

# add the time labels
for e_idx, es in enumerate(expsteps):
    meandfs[e_idx]['t'] = es
meandf = pd.concat(meandfs)

In [ ]:
plot_ifo_val(meandf, 't', ['mean', 'mean_arccos'], saveprefix='../build/out/{}{}/cs{}overtime'.format(folderprefix, expsteps[0], cellSize))

# Regnoise

In [ ]:
meandfs_noise = []
cellSize = 0.05
noises = [-2, 0., 0.1, 0.2, 0.25, 0.5, 0.75, 1.0, 1.5, 2., 5.]
foldername = 'MA_hd_t1e-7_regnoise'
folderpath = '../build/out/{}/plots'.format(foldername)
meshestxt = folderpath + '/meshes.txt'

for noisescale in noises:
    r_pref_cs = 'OriginSphere{}_{:6f}'.format(cellSize, noisescale)
    cands = [fl for fl in os.listdir(folderpath) if (r_pref_cs in fl) and ('heatgradients_vert' in fl)]
    # print(cands)
    for cand in cands:
        candname = cand.split('_vertices.csv')[0]
        # regnoise = float(cand.split('_')[1])
        rep_ind  = int(cand.split('_')[2][1:])
        #print('---> {}'.format(candname))
        #print(regnoise, rep_ind)
        
        meshesprefix = folderpath + '/OriginSphere{}_{:6f}_R{}_heatgradients'.format(cellSize, noisescale, rep_ind)
        cell_metrics_file = folderpath + '/OriginSphere{}_{:6f}_R{}_metrics.csv'.format(cellSize, noisescale, rep_ind)
        if noisescale == -2:
            cell_metrics_file = folderpath + '/OriginSphere{}_metrics.csv'.format(cellSize, noisescale, rep_ind)
        cell_metrics = pd.read_csv(cell_metrics_file)
        
        mean_delaunay = cell_metrics['delaunayflag'].mean()
        mean_contcc   = cell_metrics['containsccflag'].mean()
        
        rn_df = compare_heat_gradients_cv(meshesprefix, plot=False, outpt=False)
        N = rn_df.shape[0]
        rn_df['rep_ind']       = [rep_ind]       * N
        rn_df['regnoise']      = [noisescale]    * N
        rn_df['mean_delaunay'] = [mean_delaunay] * N
        rn_df['mean_ccc']      = [mean_contcc]   * N
        
        # print('rep_ind: {}'.format(rep_ind))
        # display(rn_df)
        
        meandfs_noise.append(rn_df)

    plot_meshesprefix = folderpath + '/OriginSphere{}_{:6f}_R0_heatgradients'.format(cellSize, noisescale)
    saveprefix='../build/out/{}/heatscatters_rn{}_cs{}'.format(foldername, noisescale, cellSize)
    compare_heat_gradients_cv(plot_meshesprefix, True, True, saveprefix=saveprefix)
    
# add the noise labels
#for n_idx, noisescale in enumerate(noises):
#    meandfs_noise[n_idx]['regnoise'] = noisescale
meandf_noise = pd.concat(meandfs_noise)

In [ ]:
def plot_regvals(df, m, an, ax, fntsz=None):
        for l_idx, ln in enumerate(df['laplacian'].unique()):
            dat = df[(df['anchor'] == an) & (df['laplacian'] == ln)]
            dat['plotx'] = dat['mean_delaunay']
            dat['plotx'].loc[dat['regnoise'] == -2.] = 1.2
            xs = dat['plotx']
            ys = dat[m]
            ax.scatter(xs, ys, c=clrs[l_idx], label='{}'.format(ln), s=70, marker='x')
        ax.set_xlim([1.3, 0.2])

        xtanchors = [1.2, 1.0, 0.8, 0.6, 0.4, 0.2]
        opt_mdval = str(dat['mean_delaunay'][dat['regnoise'] == -2.].iloc[0])[:4]
        xtlabels  = ['Optimized\n({})'.format(opt_mdval), 'Delaunay\n(1.0)'] + \
                    xtanchors[2:]
        ax.set_xticks(xtanchors)
        ax.set_xticklabels(xtlabels, rotation=90, fontsize=fntsz)
        ax.set_ylabel(metric_names[m] if m in metric_names.keys() else m, fontsize=fntsz)
        ax.set_xlabel('Fraction of Delaunay Tets', fontsize=fntsz)
        ax.axvline(x=1., lw=0.4, c='b', ls='--')
        ax.legend(fontsize=fntsz)
        ax.set_title('{}-Based'.format(an.capitalize()))

def regnoise_plot(df, metrics=None, saveprefix=None, usedelaunayxaxis=True, both=False, fntsz=14):
    lss  = ['-', '--']
    index_cols = ['anchor', 'laplacian', 'regnoise']
    axis_descriptors = {'t' : 'log(t)'}
    mean_delaunay_vals = df.groupby('regnoise')['mean_delaunay'].mean()
    
    if metrics is None:
        metrics = [m for m in df.keys() if m not in index_cols]
        
    for m in metrics:
        rows = 1
        cols = 2 if both else 1
        fig, axs = plt.subplots(1, rows, figsize=(10 if both else 7, 5), sharey=True)
        
        plot_regvals(df, m, 'vertex', axs[0] if both else axs, fntsz=fntsz)
        if both:
            plot_regvals(df, m, 'cell', axs[1], fntsz=fntsz)
            
        fig.subplots_adjust(bottom=0.3)
        # plt.tight_layout()
        if saveprefix is not None:
            plt.savefig(saveprefix + m + '.png', dpi=300)
        plt.show()

In [ ]:
mean_delaunay_vals = meandf_noise.groupby('regnoise')['mean_delaunay'].mean()
regnoise_plot(meandf_noise, ['mean_arccos', 'mean'], os.path.join(folderpath, 'cs{}regnoise_plot'.format(cellSize)), False)

# Inspect the scatters: negative values

In [ ]:
base_folder = '../build/out/LA_hd_t1e-7/plots/'
mesh_prefix = 'OriginSphere{}'.format(0.1)
dd_mesh_prefix = '{}_0.000000_R0'.format(mesh_prefix)
nd_mesh_prefix = '{}_-2.000000_R0'.format(mesh_prefix)
_=compare_heat_gradients_cv(base_folder + nd_mesh_prefix + '_heatgradients', plot=True)
_=compare_heat_gradients_cv(base_folder + dd_mesh_prefix + '_heatgradients', plot=True)

### Are the bad ones the nd ones?

In [ ]:
names, anchors_cells, products_cells = load_heat_gradients(base_folder + nd_mesh_prefix + '_heatgradients_cells.csv')
cell_metrics = pd.read_csv(base_folder + nd_mesh_prefix + '_metrics.csv' )

In [ ]:
print(cell_metrics.shape)
print(products_cells[0].shape)

nd_indices = cell_metrics['delaunayflag'] == 0

print('FEM')
print(products_cells[0].mean())
print(products_cells[0][nd_indices].mean())

print('DEC')
print(products_cells[1].mean())
print(products_cells[1][nd_indices].mean())

print('REG')
print(products_cells[2].mean())
print(products_cells[2][nd_indices].mean())

display(nd_indices.sum())
print((products_cells[1] < 0).sum())

In [ ]:
bad_cell_indices_dec  = (products_cells[1] < -0.6)
good_cell_indices_dec = (products_cells[1] > 0)
for m in cell_metrics.keys():
    print('{}: '.format(m))
    print('Good Cells: {}'.format(cell_metrics[good_cell_indices_dec][m].mean()))
    print('Bad  Cells: {}'.format(cell_metrics[bad_cell_indices_dec][m].mean()))

### Vertices, matrix:

In [ ]:
from scipy.io import mmread
def load_laplacian(path):
     # LAPLACE STATS:
    L = mmread(path).toarray()
    return L

In [ ]:
names, anchors_verts, products_verts = load_heat_gradients(base_folder + nd_mesh_prefix + '_heatgradients_vertices.csv')
Ldec = load_laplacian(base_folder + nd_mesh_prefix + '__Ldec.mtx')
Lfem = load_laplacian(base_folder + nd_mesh_prefix + '__Lfem.mtx')

In [ ]:
def angle_entry_scatter(products, L, ax=None):
    L[np.diag_indices(L.shape[0])] = 0.
    nzs_inds = L != 0.
    xs = (products[:,None] * np.ones(products.shape[0])[None,:])[nzs_inds]
    ys = L[nzs_inds]
    ps = ys >= 0.
    ns = ys <  0.
    
    # presentation
    if ax is None:
        ax = plt.gca()
    ax.scatter(xs[ps], ys[ps], s=0.5, c='g')
    ax.scatter(xs[ns], ys[ns], s=0.5, c='r')
    ax.set_xlabel('dot prod (cos angle)')
    ax.set_ylabel('laplacian weights')

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 4), sharey=True)
angle_entry_scatter(products_verts[0], Ldec, ax=axs[0])
axs[0].set_title('FEM')
angle_entry_scatter(products_verts[1], Lfem, ax=axs[1])
axs[1].set_title('DEC')
fig.suptitle('Laplacian weights ifo angle')
plt.show()

### Weighted Points?

In [ ]:
base_folder = '../build/out/LA_hd_t1e-7/plots/'
mesh_prefix = 'OriginSphere{}'.format(0.1)
dd_mesh_prefix = '{}_0.000000_R0'.format(mesh_prefix)
nd_mesh_prefix = '{}_-2.000000_R0'.format(mesh_prefix)
names, anchors_verts, products_verts = load_heat_gradients(base_folder + nd_mesh_prefix + '_heatgradients_vertices.csv')
regweights = pd.read_csv(base_folder + mesh_prefix + '__regweights.csv')

fig, axs = plt.subplots(1, 2, figsize=(10, 4), sharey=True)
axs[0].scatter(products_verts[0], list(regweights['regweight']), s=0.5, c=clrs[0])
axs[0].set_title('FEM')
axs[0].set_xlabel('cos(angle)')
axs[0].set_ylabel('vertex weight')

axs[1].scatter(products_verts[1], list(regweights['regweight']), s=0.5, c=clrs[1])
axs[1].set_title('DEC')
axs[1].set_xlabel('cos(angle)')

fig.suptitle('Vertex weights (regular tri) ifo angle')
plt.show()

# Negative Values and different values to reg:

In [ ]:
def scatter_nonzero_entries_along_val(L, val):
    nv = L.shape[0]
    nzro = L != 0
    xs = (val[:,None] * np.ones(nv)[None,:])[nzro]
    ys = L[nzro]
    plt.scatter(xs, ys, s=0.5)
    plt.show()

In [ ]:
base_folder = '../build/out/LA_hd_t1e-7/plots/'
mesh_prefix = 'OriginSphere{}'.format(0.1)
dd_mesh_prefix = '{}_0.000000_R0'.format(mesh_prefix)
nd_mesh_prefix = '{}_-2.000000_R0'.format(mesh_prefix)
Ldec = load_laplacian(base_folder + nd_mesh_prefix + '__Ldec.mtx')
Lreg = load_laplacian(base_folder + mesh_prefix + '_decregLoptimized.mtx')
regweights = np.asarray(pd.read_csv(base_folder + mesh_prefix + '__regweights.csv')['regweight'])

In [ ]:
Ldiff = (Lreg - Ldec)
scatter_nonzero_entries_along_val(Ldiff, regweights)

In [ ]:
plt.scatter(np.linalg.norm(anchors_verts, axis=1), regweights, s=0.5)
plt.show()

In [ ]:
names, anchors_verts, products_verts = load_heat_gradients(base_folder + nd_mesh_prefix + '_heatgradients_vertices.csv')
Ldiff = (Lreg - Ldec)
scatter_nonzero_entries_along_val(Ldiff, np.linalg.norm(anchors_verts, axis=1))